This tutorial can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2026.1_tutorials/wallaroo-run-anywhere/inference/arm/wallaroo-arm-byop-vgg16).

## Custom Model ARM Deployment Tutorial

This tutorial demonstrates how to use Wallaroo Custom Model as a ML Model in Wallaroo.  Custom Model allows organizations to use Python scripts that require specific libraries and artifacts as models in the Wallaroo engine.  This allows for highly flexible use of ML models with supporting scripts.

### Custom Model Script Requirements

The entry point of the Wallaroo Custom Model aka BYOP (Bring Your Own Predict) model is any python script that **must** include the following.

* `class ImageClustering(Inference)`:  The default inference class.  This is used to perform the actual inferences.  Wallaroo uses the `_predict` method to receive the inference data and call the appropriate functions for the inference.
  * `def __init__`:  Used to initialize this class and load in any other classes or other required settings.
  * `def expected_model_types`: Used by Wallaroo to anticipate what model types are used by the script.
  * `def model(self, model)`: Defines the model used for the inference.  Accepts the model instance used in the inference.
    * `self._raise_error_if_model_is_wrong_type(model)`: Returns the error if the wrong model type is used.  This verifies that only the anticipated model type is used for the inference.
    * `self._model = model`: Sets the submitted model as the model for this class, provided `_raise_error_if_model_is_wrong_type` is not raised.
  * `def _predict(self, input_data: InferenceData)`:  This is the entry point for Wallaroo to perform the inference.  This will receive the inference data, then perform whatever steps and return a dictionary of numpy arrays.
* `class ImageClusteringBuilder(InferenceBuilder)`: Loads the model and prepares it for inferencing.
  * `def inference(self) -> ImageClustering`: Sets the inference class being used for the inferences.
  * `def create(self, config: CustomInferenceConfig) -> ImageClustering`: Creates an inference subclass, assigning the model and any attributes required for it to function.

All other methods used for the functioning of these classes are optional, as long as they meet the requirements listed above.

### Tutorial Prerequisites

* Wallaroo Version 2023.2.1 or above instance.

### References

* [Wallaroo SDK Essentials Guide: Model Uploads and Registrations: Custom Model](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/wallaroo-sdk-model-arbitrary-python/)

## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  These are included by default in the Wallaroo instance's JupyterHub service.

In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import wallaroo

from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more details on logging in through Wallaroo, see the [Wallaroo SDK Essentials Guide: Client Connection](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
wl = wallaroo.Client()



### Set Variables

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.



In [3]:
workspace_name = f'vgg16-clustering-workspace-error-tests'
pipeline_name = f'vgg16-clustering-error-tests'

model_name = 'vgg16-clustering-error-tests'
model_file_name = './models/model-auto-conversion-BYOP-vgg16-clustering.zip'

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline that is used to deploy our arbitrary Python model.

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)
wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

### Upload Custom Model

Custom Models are uploaded to Wallaroo through the Wallaroo Client `upload_model` method.

#### Upload Custom Model Parameters

The following parameters are required for Custom Models.  Note that while some fields are considered as **optional** for the `upload_model` method, they are required for proper uploading of a Custom Model to Wallaroo.

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. |
|`framework` |`string` (*Upload Method Optional, Custom Model Required*) | Set as `Framework.CUSTOM`. |
|`input_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, Custom Model Required*) | The input schema in Apache Arrow schema format. |
|`output_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, Custom Model Required*) | The output schema in Apache Arrow schema format. |
| `convert_wait` | `bool` (*Upload Method Optional, Custom Model Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

#### Upload Custom Model Return

The following is returned with a successful model upload and conversion.

| Field | Type | Description |
|---|---|---|
| `name` | string | The name of the model. |
| `version` | string | The model version as a unique UUID. |
| `file_name` | string | The file name of the model as stored in Wallaroo. |
| `image_path` | string | The image used to deploy the model in the Wallaroo engine. |
| `last_update_time` | DateTime | When the model was last updated. |

For our example, we'll start with setting the `input_schema` and `output_schema` that is expected by our `ImageClustering._predict()` method.

In [5]:
input_schema = pa.schema([
    pa.field('images', pa.list_(
        pa.list_(
            pa.list_(
                pa.int64(),
                list_size=3
            ),
            list_size=32
        ),
        list_size=32
    )),
])

output_schema = pa.schema([
    pa.field('predictions', pa.int64()),
])

### Upload Model

Now we'll upload our model.  The framework is `Framework.CUSTOM` for arbitrary Python models, and we'll specify the input and output schemas for the upload.

An important note for the ARM version of the VGG16 model is verifying that the Python modules used will operate in an ARM environment.  For example, the x86 model has the following as its `requirements.txt`:

```python
tensorflow==2.8.0
scikit-learn==1.2.2
```

The ARM version replaces the `tensorflow` module with the `tensorflow-aarch64` that runs in the ARM archtecture.  Otherwise, the model and Python script is the same.

```python
tensorflow-aarch64==2.8.4
scikit-learn==1.2.2
```

Note that during the upload, the `arch` setting is `wallaroo.engine_config.Architecture.ARM`.

In [6]:
from wallaroo.engine_config import Architecture

model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=Framework.CUSTOM, 
                        input_schema=input_schema, 
                        output_schema=output_schema,
                        convert_wait=True)
model

Waiting for model loading - this will take up to 10min.
Model is pending loading to a container runtime.
Model is attempting loading to a container runtime.......
Successful
Ready


Name,vgg16-clustering-error-tests
Version,92e5a0ab-5692-4172-883f-ce056e682af5
File Name,model-auto-conversion-BYOP-vgg16-clustering.zip
SHA,79472d4b9652937c15e1887478c23effe1160363b985920e083fec72868810f4
Status,ready
Image Path,proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mac-deploy:v2025.2.0-6480
Architecture,x86
Acceleration,none
Updated At,2025-18-Dec 16:23:55
Workspace id,1813
Workspace name,vgg16-clustering-workspace-error-tests


In [7]:
print(model)

{'name': 'vgg16-clustering-error-tests', 'version': '92e5a0ab-5692-4172-883f-ce056e682af5', 'file_name': 'model-auto-conversion-BYOP-vgg16-clustering.zip', 'image_path': 'proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mac-deploy:v2025.2.0-6480', 'arch': 'x86', 'accel': 'none', 'last_update_time': datetime.datetime(2025, 12, 18, 16, 23, 55, 121001, tzinfo=tzutc())}


### Deploy Pipeline

The model is uploaded and ready for use.  We'll add it as a step in our pipeline, then deploy the pipeline.  For this example we're allocated 0.25 cpu and 4 Gi RAM to the pipeline through the pipeline's deployment configuration.

The deployment configuration inherits its Architecture setting from the model, so it will automatically be set for `ARM` processors.

In [8]:
pipeline.clear()
pipeline.add_model_step(model)

name,vgg16-clustering-error-tests
created,2025-12-18 16:22:55.352014+00:00
last_updated,2025-12-18 16:22:55.352014+00:00
deployed,(none)
workspace_id,1813
workspace_name,vgg16-clustering-workspace-error-tests
arch,None
accel,none
tags,
versions,db233e12-1082-4a81-9745-ef2f55d793fe
steps,


In [9]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(1).memory('4Gi') \
    .build()

pipeline.deploy(deployment_config=deployment_config, wait_for_status=False)

Deployment initiated for vgg16-clustering-error-tests. Please check pipeline status.


name,vgg16-clustering-error-tests
created,2025-12-18 16:22:55.352014+00:00
last_updated,2025-12-18 16:24:02.327865+00:00
deployed,True
workspace_id,1813
workspace_name,vgg16-clustering-workspace-error-tests
arch,x86
accel,none
tags,
versions,"11e8fcf7-5a72-4acc-b93c-e1435b67bfb1, db233e12-1082-4a81-9745-ef2f55d793fe"
steps,vgg16-clustering-error-tests


In [10]:
pipeline.status()

{'status': 'Starting',
 'details': ['Scaling'],
 'engines': [{'ip': None,
   'name': 'engine-646b859d6b-vhn7z',
   'status': 'Pending',
   'reason': None,
   'details': ['containers with unready status: [engine]',
    'containers with unready status: [engine]'],
   'pipeline_statuses': None,
   'model_statuses': None}],
 'engine_lbs': [{'ip': None,
   'name': 'engine-lb-7f8cdcd455-hbsbs',
   'status': 'Pending',
   'reason': None,
   'details': ['containers with unready status: [engine-lb]',
    'containers with unready status: [engine-lb]']}],
 'sidekicks': [{'ip': None,
   'name': 'engine-sidekick-vgg16-clustering-error-tests-1132-8569ccf5949cn',
   'status': 'Pending',
   'reason': None,
   'details': ['containers with unready status: [engine-sidekick-vgg16-clustering-error-tests-1132]',
    'containers with unready status: [engine-sidekick-vgg16-clustering-error-tests-1132]'],
   'statuses': None}]}

In [11]:
import time
time.sleep(15)

while pipeline.status()['status'] != 'Running':
    time.sleep(15)
    print("Waiting for deployment.")
    display(pipeline.status()['status'])
pipeline.status()['status']


Waiting for deployment.


'Starting'

Waiting for deployment.


'Running'

'Running'

### Run inference

Everything is in place - we'll now run a sample inference with some toy data.  In this case we're randomly generating some values in the data shape the model expects, then submitting an inference request through our deployed pipeline.

In [12]:
input_data = {
        "images": [np.random.randint(0, 256, (32, 32, 3), dtype=np.uint8)] * 2,
}
dataframe = pd.DataFrame(input_data)
dataframe

,images
0,"[[[81, 205, 112], [15, 218, 108], [245, 84, 23..."
1,"[[[81, 205, 112], [15, 218, 108], [245, 84, 23..."


In [13]:
pipeline.infer(dataframe, timeout=10000)

,time,in.images,out.predictions,anomaly.count
0,2025-12-18 16:25:26.599,"[[[81, 205, 112], [15, 218, 108], [245, 84, 23...",1,0
1,2025-12-18 16:25:26.599,"[[[81, 205, 112], [15, 218, 108], [245, 84, 23...",1,0


### Undeploy Pipelines

The inference is successful, so we will undeploy the pipeline and return the resources back to the cluster.

In [16]:
pipeline.undeploy()

name,vgg16-clustering-error-tests
created,2025-12-18 16:22:55.352014+00:00
last_updated,2025-12-18 16:24:02.327865+00:00
deployed,False
workspace_id,1813
workspace_name,vgg16-clustering-workspace-error-tests
arch,x86
accel,none
tags,
versions,"11e8fcf7-5a72-4acc-b93c-e1435b67bfb1, db233e12-1082-4a81-9745-ef2f55d793fe"
steps,vgg16-clustering-error-tests
